### 重要知識點
* 了解何謂語言模型
* 了解 N-Gram 語言模型的原理以及應用

### 什麼是語言模型？
當我們想到可口可樂的時候，可能會”聯想”到百事可樂(而不是珍珠奶茶)，人們長期在學習累積語言後有判斷合不合理與聯想的能力。Ｎ-gram 就是基於這樣聯想的語言模型。

### 作業目標: 了解N-Gram如何藉由文本計算機率

### 為何需要使用馬可夫假設來簡化語言模型的計算?

原本的語言模型利用貝氏定理計算機率時為:
$$
\begin{aligned}
&W = (W_1W_2W_3W_4…W_m) \\
&P(W_1,W_2,W_3,W_4,…,W_m) = P(W_1)*P(W_2|W_1)*P(W_3|W_1,W_2)*…*P(W_m|W_1,…,W_{m-1})
\end{aligned}
$$

為何需要引入馬可夫假設使機率簡化為:
$$
P(W_m|W_1,W_2,W_3,…,W_{m-1}) = P(W_m|W_{m-n+1},W_{m-n+2},…,W_{m-1})
$$

In [ ]:
'''
###<your answer>###
當N值較大：對字詞的約束性更高，具有更高的辨識力，複雜度較高
(前面的句子越長，會包含更多不需要的資訊，反而會降低可能答案的命中率)

當Ｎ值較小：字詞在文本出現的次數較多，更可靠的統計結果，但對字詞的約束性較小
(因此只考慮緊鄰縮短的字詞會包含更多重要的資訊)
'''

### 以Bigram模型下判斷語句是否合理

#### 已知的機率值有
1. p(i|_start_) = 0.25
2. p(english|want) = 0.0011
3. p(food|english) = 0.5
4. p(_end_|food) = 0.68
5. P(want|_start_) = 0.25
6. P(english|i) = 0.0011

In [7]:
import numpy as np
import pandas as pd
words = ['i', 'want', 'to', 'eat', 'chinese', 'food', 'lunch', 'spend']
word_cnts = np.array([2533, 927, 2417, 746, 158, 1093, 341, 278]).reshape(1, -1)
df_word_cnts = pd.DataFrame(word_cnts, columns=words)
df_word_cnts

,i,want,to,eat,chinese,food,lunch,spend
0,2533,927,2417,746,158,1093,341,278


In [4]:
# 記錄當前字與前一個字詞的存在頻率

bigram_word_cnts = [[5, 827, 0, 9, 0, 0, 0, 2], [2, 0, 608, 1, 6, 6, 5, 1], [2, 0, 4, 686, 2, 0, 6, 211],
                    [0, 0, 2, 0, 16, 2, 42, 0], [1, 0, 0, 0, 0, 82, 1, 0], [15, 0, 15, 0, 1, 4, 0, 0],
                    [2, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0]]

df_bigram_word_cnts = pd.DataFrame(bigram_word_cnts, columns=words, index=words)
df_bigram_word_cnts

,i,want,to,eat,chinese,food,lunch,spend
i,5,827,0,9,0,0,0,2
want,2,0,608,1,6,6,5,1
to,2,0,4,686,2,0,6,211
eat,0,0,2,0,16,2,42,0
chinese,1,0,0,0,0,82,1,0
food,15,0,15,0,1,4,0,0
lunch,2,0,0,0,0,1,0,0
spend,1,0,1,0,0,0,0,0


由上表可知當前一個字詞(列)是want的時候, 當前字詞(行)是to的頻率在文本中共有608次

* print(df_bigram_prob.loc[word, :]) # 選取每一列word的資料
* print(df_word_cnts.loc[0, word]) # 總字頻
* P(i want to) = P(i)*P(want|i)*P(to|want)

       i        want        to      eat      chinese   food   lunch   spend 
i   0.001974   0.32649     0.0    0.003553      0.0     0.0    0.0    -----> 每一個都除以2533(i出現的總次數＝前一個字是i的機率)

In [15]:
#請根據給出的總詞頻(df_word_cnts)與bigram模型的詞頻(df_bigram_word_cnts)計算出所有詞的配對機率(ex:p(want|i))
df_bigram_prob = df_bigram_word_cnts.copy()
###<your code>###
for word in words:
    df_bigram_prob.loc[word, :] = df_bigram_prob.loc[word, :] / df_word_cnts.loc[0, word]

df_bigram_prob

,i,want,to,eat,chinese,food,lunch,spend
i,0.001974,0.32649,0.000000,0.003553,0.000000,0.000000,0.000000,0.000790
want,0.002157,0.00000,0.655879,0.001079,0.006472,0.006472,0.005394,0.001079
to,0.000827,0.00000,0.001655,0.283823,0.000827,0.000000,0.002482,0.087298
eat,0.000000,0.00000,0.002681,0.000000,0.021448,0.002681,0.056300,0.000000
chinese,0.006329,0.00000,0.000000,0.000000,0.000000,0.518987,0.006329,0.000000
food,0.013724,0.00000,0.013724,0.000000,0.000915,0.003660,0.000000,0.000000
lunch,0.005865,0.00000,0.000000,0.000000,0.000000,0.002933,0.000000,0.000000
spend,0.003597,0.00000,0.003597,0.000000,0.000000,0.000000,0.000000,0.000000


請根據已給的機率與所計算出的機率(df_bigram_prob), 試著判斷下列兩個句子哪個較為合理

s1 = “i want english food”

s2 = "want i english food"

In [19]:
'''
###<your answer>###
s1 = 0.25*0.32649*0.0011*0.5*0.68
s2 = 0.25*0.002157*0.0011*0.5*0.68
'''

s1 = 0.25*0.32649*0.0011*0.5*0.68
print(f's1的機率為{s1}')
s2 = 0.25*0.002157*0.0011*0.5*0.68
print(f's2的機率為{s2}')

if s1>s2 :
    print("所以s1較合理")
else:
    print("所以s2較合理")

s1的機率為3.0526815e-05
s2的機率為2.0167950000000003e-07
所以s1較合理


In [ ]:
### 以Bigram模型下判斷語句是否合理

#### 已知的機率值有
1. p(i|_start_) = 0.25
2. p(english|want) = 0.0011
3. p(food|english) = 0.5
4. p(_end_|food) = 0.68
5. P(want|_start_) = 0.25
6. P(english|i) = 0.0011